In [3]:
import sympy as sy
import numpy as np

In [5]:
x1, mx1, x2, mx2 = sy.var('x1, mu_x1, x2, mu_x2', real=True)
a, b = sy.var('a, b', positive=True, integer=True)

In [6]:
def m_ab(x1, a, x2, b):
    """Co-moment of x1**a and x2**b""" 
    return sy.expand((x1 - mx1)**a * (x2 - mx2)**b)

In [7]:
# An version of the expression that uses stohastic vars. `E` is handled as a linear function,
# but in our case it does not help much as expectations have only one argument each.
# Also ideally r and t should be declared as sample valaues to get complete expression (top level Sum).
from sympy.stats import P, E, variance, Die, Normal
from sympy.stats.rv import RandomSymbol
mr, dr = sy.var('mu_r, dr')
mt, dt = sy.var('mu_t, dt')
r = RandomSymbol('r')
t = RandomSymbol('t')
sy.expand(E((r - E(r))**2*(t - E(t))**3)) \
    .replace(E(r), mr)  \
    .replace(E(t), mt)
# There seem to be no standard way to sort members by powers (sp.Poly does print them in a
# consistent way but that order is lost when the sum expression is extracted from it)

-mu_r**2*mu_t**3 + 3*mu_r**2*mu_t**2*t - 3*mu_r**2*mu_t*t**2 + mu_r**2*t**3 + 2*mu_r*mu_t**3*r - 6*mu_r*mu_t**2*r*t + 6*mu_r*mu_t*r*t**2 - 2*mu_r*r*t**3 - mu_t**3*r**2 + 3*mu_t**2*r**2*t - 3*mu_t*r**2*t**2 + r**2*t**3

In [8]:
# This version extracts common expressions, but in our case it does not seem to be a significant optimization.
# Need to benchmark it to see the impact.
(common_vals, exprs) =  sy.cse(sy.simplify(m_ab(x1, 2, x2, 4)))
(common_vals, exprs)

([(x0, mu_x1**2),
  (x1, mu_x2**4),
  (x2, x2**4),
  (x3, x1**2),
  (x4, mu_x1*x1),
  (x5, 2*x4),
  (x6, mu_x2*x2**3),
  (x7, 8*x4),
  (x8, mu_x2**3*x2),
  (x9, 4*x0),
  (x10, 4*x3),
  (x11, mu_x2**2*x2**2),
  (x12, 6*x11)],
 [x0*x1 + x0*x12 + x0*x2 + x1*x3 - x1*x5 - x10*x6 - x10*x8 - 12*x11*x4 + x12*x3 + x2*x3 - x2*x5 + x6*x7 - x6*x9 + x7*x8 - x8*x9])

In [9]:
# An example creating a callable function in sympy. It requires sympy dependency and will have to 
# have a parameter for each member (x1^a1*x2^b2).
# https://docs.sympy.org/latest/modules/utilities/lambdify.html
import inspect
lf = sy.lambdify((x1, mx1, x2, mx2), sy.simplify(m_ab(x1, 2, x2, 4)), "numpy")
print(inspect.getsource(lf))
lf(23, 21, 1.1, 1.3)

def _lambdifygenerated(x1, mu_x1, x2, mu_x2):
    return mu_x1**2*mu_x2**4 - 4*mu_x1**2*mu_x2**3*x2 + 6*mu_x1**2*mu_x2**2*x2**2 - 4*mu_x1**2*mu_x2*x2**3 + mu_x1**2*x2**4 - 2*mu_x1*mu_x2**4*x1 + 8*mu_x1*mu_x2**3*x1*x2 - 12*mu_x1*mu_x2**2*x1*x2**2 + 8*mu_x1*mu_x2*x1*x2**3 - 2*mu_x1*x1*x2**4 + mu_x2**4*x1**2 - 4*mu_x2**3*x1**2*x2 + 6*mu_x2**2*x1**2*x2**2 - 4*mu_x2*x1**2*x2**3 + x1**2*x2**4



0.006399999999757711

In [10]:
# The method that lambdify uses. Does not do any namespace subtitutions, though 
# (like adapting/converting operations to numpy or scypy).
from sympy.utilities.lambdify import lambdastr
lambdastr([x1, mx1, x2, mx2], sy.simplify(m_ab(x1, 2, x2, 4)))

'lambda x1,mu_x1,x2,mu_x2: (mu_x1**2*mu_x2**4 - 4*mu_x1**2*mu_x2**3*x2 + 6*mu_x1**2*mu_x2**2*x2**2 - 4*mu_x1**2*mu_x2*x2**3 + mu_x1**2*x2**4 - 2*mu_x1*mu_x2**4*x1 + 8*mu_x1*mu_x2**3*x1*x2 - 12*mu_x1*mu_x2**2*x1*x2**2 + 8*mu_x1*mu_x2*x1*x2**3 - 2*mu_x1*x1*x2**4 + mu_x2**4*x1**2 - 4*mu_x2**3*x1**2*x2 + 6*mu_x2**2*x1**2*x2**2 - 4*mu_x2*x1**2*x2**3 + x1**2*x2**4)'

In [59]:
# ScyPy matrices https://docs.sympy.org/latest/tutorials/intro-tutorial/matrices.html

In [48]:
nv = 5
na = 2
nb = 3
powers = [[0] * nb] * na
for a in range(na):
    for b in range(nb):
        powers[a][b] = np.zeros((nv, nv))
# print(powers)

In [93]:
sample = np.random.normal(1, 2, size=(33, nv)) # Samples in rows
Xt = sample.T
X = sample
for a in range(na):
    Xt X = np.matmul(X.T, X)
    for b in range(nb):
        assert XX.shape == (nv, nv)
        powers[a][b] = XX
    
print(XX)

[[145.15854404  -4.19695487  -2.76441156  -2.70612867  24.8616544 ]
 [ -4.19695487 153.5124429   26.33765884   9.57634836  48.85337473]
 [ -2.76441156  26.33765884 127.57002124  41.54439012  84.82353029]
 [ -2.70612867   9.57634836  41.54439012 111.19425501  53.18970077]
 [ 24.8616544   48.85337473  84.82353029  53.18970077 149.29342026]]


array([[ 2.6759676 ,  2.90212223,  1.42116702,  3.19111171, -0.17306954],
       [ 1.35639147,  1.05314554,  0.27627967, -1.13525339, -0.99277624],
       [-0.92769366, -0.47412928,  3.58729789,  4.22027257, -0.87973768],
       [ 3.29083784,  2.1147043 , -1.10866322, -2.36153713,  3.07037793]])

In [79]:
X.T

array([[ 2.6759676 ,  1.35639147, -0.92769366,  3.29083784],
       [ 2.90212223,  1.05314554, -0.47412928,  2.1147043 ],
       [ 1.42116702,  0.27627967,  3.58729789, -1.10866322],
       [ 3.19111171, -1.13525339,  4.22027257, -2.36153713],
       [-0.17306954, -0.99277624, -0.87973768,  3.07037793]])